In [3]:
##function:load the data and delete the abnormal data
def load(path):
    # coding:utf-8
    # read the csv
    data_raw = []
    with open(path,encoding='UTF-8') as file:
        for line in file.readlines():
            line = line.split('"')
            l = []
            for i in line:
                if i != '' and i != ',' and i != '\n':
                    l.append(i)
            data_raw.append(l)

    #delete the same data
    data = []
    for i in data_raw:
        if not i in data:
            data.append(i)
    
    #get the title of the data
    global title,title_con,title_dis
    title = data[0]
    title_con = ['Reviews','Size','Installs','Price']
    title_dis = ['Category','Content.Rating','Genres']

    #delete the title
    del(data[0])
    for i in range(0,len(data)):
        for j in range(0,len(data[i])):
            data[i][j] = data[i][j].strip(',')
            data[i][j] = data[i][j].strip()

    #abnormal deletion and data transformation
    abnormal = []
    for i in range(0,len(data)):
        #for rating
        if data[i][2] != 'NA':    
            try:
                data[i][2] = float(data[i][2])
            except ValueError:
                abnormal.append(data[i])
        else:
            abnormal.append(data[i])

        #for reviews
        if data[i][3] != 'NA':
            try:
                data[i][3] = float(data[i][3])
            except ValueError:
                abnormal.append(data[i])
        else:
            abnormal.append(data[i])

        #for Size
        if data[i][4][-1] == 'k':
            try:
                data[i][4] = float(data[i][4].strip('k'))
            except ValueError:
                abnormal.append(data[i])
        elif data[i][4][-1] == 'M':
            try:
                data[i][4] = float(data[i][4].strip('M'))*1024
            except ValueError:
                abnormal.append(data[i])
        else:
            abnormal.append(data[i])

        #for Installs and Price
        try:
            data[i][5] = float(data[i][5].strip('+').replace(',',''))
            data[i][7] = float(data[i][7].strip('$'))
        except ValueError:
            abnormal.append(data[i])

    data_new = []  
    for i in data:
        if i not in abnormal and len(i) == 13:
            data_new.append(i)

    return data_new

In [4]:
##function:dividing data for continuous data
def div_con(data,feature,value):
    left = []
    right = []
    for i in range(0,len(data)):
        if data[i][feature] <= value:
            left.append(data[i])
        else:
            right.append(data[i])
    return left,right

##function:finding the probably value for a continuous feature
def find_feature_con(data,feature):
    value = []
    probable_value = []
    for i in range(0,len(data)):
        if data[i][feature] not in value:
            value.append(data[i][feature])
    value.sort()
    for i in range(0,len(value)-1):
        probable_value.append((value[i]+value[i+1])/2)
    return probable_value

##function:dividing data for discrete data
def div_dis(data,feature,value):
    left = []
    right = []
    for i in range(0,len(data)):
        if (data[i][feature] == value):
            left.append(data[i])
        else:
            right.append(data[i])
    return left,right  

##function:finding the probably value for a discrete feature
def find_feature_dis(data,feature):
    sets = []
    for i in range(0,len(data)):
        if data[i][feature] not in sets:
            sets.append(data[i][feature])
    return sets

##function:calculating the Gini index
def gini_index(left,right):
    p1 = 0
    p2 = 0
    for i in left:
        if i[2] > 4.5:
            p1 = p1+1
        else:
            p2 = p2+1
    if p1 == 0 or p2 == 0:
        Gini_left = 0
    else:
        Gini_left = 1 - (p1/(p1+p2))**2 - (p2/(p1+p2))**2
    
    p1 = 0
    p2 = 0
    for i in right:
        if i[2] > 4.5:
            p1 = p1+1
        else:
            p2 = p2+1
    if p1 == 0 or p2 == 0:
        Gini_right = 0
    else:
        Gini_right = 1 - (p1/(p1+p2))**2 - (p2/(p1+p2))**2
        
    Gini_D = len(left)/(len(left)+len(right)) * Gini_left + len(right)/(len(left)+len(right)) * Gini_right
    return Gini_D

##function:generating the leaf:

def gen_leaf(data):
    positive = 0
    negative = 0
    
    #if it is a leaf node, use the value with larger percent to predict the value
    for i in range(0,len(data)):
        if data[i][2] > 4.5:
            positive = positive + 1
        else:
            negative = negative + 1
    if positive >= negative:
        return 'positive'
    else:
        return 'negative'

In [5]:
##function:finding the best features and the best dividing point
##input:
##    data : the data set to find the best feature and best value
##output:
##    best_feature : best feature to divide
##    best_value : best value to divide in best feature

def find_feature(data):
    
    #if the label is the same for all data, then return leaf node
    label = []
    for i in data:
        if i[2] > 4.5:
            label.append('positive')
        else:
            label.append('negative')
    if len(set(label)) == 1:
        return None,gen_leaf(data)
    
    #get the gini index for the orginal data
    p1 = 0
    p2 = 0
    for i in range(0,len(data)):
        if data[i][2]> 4.5:
            p1 = p1+1
        else:
            p2 = p2+1
    original_Gini = 1 - (p1/(p1+p2))**2 - (p2/(p1+p2))**2
    
    #initialize the value
    best_Gini = float('inf')
    best_feature = None
    best_value = None
    
    
    #try all the features and all the dividing points to determine the best feature
    #for continuous variables
    for i in title_con:
        feature = title.index(i)
        probable_value = find_feature_con(data,feature)
        for value in probable_value:
            left,right = div_con(data,feature,value)
            Gini_D = gini_index(left,right)
            if Gini_D < best_Gini:
                best_Gini = Gini_D
                best_feature = feature
                best_value = value
    
    #for discrete variables
    for i in title_dis:
        feature = title.index(i)
        sets = find_feature_dis(data,feature)
        for value in sets:
            left,right = div_dis(data,feature,value)
            Gini_D = gini_index(left,right)
            if Gini_D < best_Gini:
                best_Gini = Gini_D
                best_feature = feature
                best_value = value
    
    #if gini index after classification is larger than before, then return leaf node
    if best_Gini > original_Gini:
        return None,gen_leaf(data)
    
    #get the data sets after classification
    if title[best_feature] in title_con:
        left,right = div_con(data,best_feature,best_value)
    else:
        left,right = div_dis(data,best_feature,best_value)
    
    #if data sets after classification is too small, then return leaf node
    if len(left) < 5 or len(right) < 5:
        return None,gen_leaf(data)
    
    #otherwise, return best feature and best value
    return best_feature,best_value

In [6]:
##function: create CART
##input:
##    data : data for generating trees
##output:
##    tree : a dictionary that indicates the CART
##for continuous data, left means <= value
##for discrete data, left means = value

def create_tree(data):
    best_feature,best_value = find_feature(data)
    #if generate leaf node, return the value of leaf
    if best_feature == None:
        return best_value
    
    tree = {}
    tree['feature'] = title[best_feature]
    tree['value'] = best_value
    if title[best_feature] in title_con:
        left,right = div_con(data,best_feature,best_value)
    else:
        left,right = div_dis(data,best_feature,best_value)
    
    #recursion to generate tree
    tree['left'] = create_tree(left)
    tree['right'] = create_tree(right)
    
    return tree

In [7]:
##function:test data using the tree

def test_tree(test,tree):
    
    #for continuous value
    if tree['feature'] in title_con:   
        #for left side
        if test[title.index(tree['feature'])] <= tree['value']:
            #if it is not a leaf node, then recursion
            if isinstance(tree['left'],dict):
                return test_tree(test,tree['left'])
            #else return the value
            else:                
                return tree['left']
            
        #for right side
        else:
            #if it is not a leaf node, then recursion
            if isinstance(tree['right'],dict):
                return test_tree(test,tree['right'])
            #else return the value
            else:
                return tree['right']
        
    #for discrete value
    else:   
        #for left side
        if test[title.index(tree['feature'])] == tree['value']:
            #if it is not a leaf node, then recursion
            if isinstance(tree['left'],dict):
                 return test_tree(test,tree['left'])
            #else return the value
            else:                
                return tree['left']
            
        #for right side
        else:
            #if it is not a leaf node, then recursion
            if isinstance(tree['right'],dict):
                return test_tree(test,tree['right'])
            #else return the value
            else:
                return tree['right']

In [8]:
##function:compute the accuracy

def test(data,tree):
    predict = []
    real = []
    accurate = 0
    
    #predict for each line and label for the test data
    for i in data:
        test = i
        pre = test_tree(test,tree)
        predict.append(pre)   
        re = test[2]
        if re > 4.5:
            real.append('positive')
        else:
            real.append('negative')
            
    #if the predict = real, accurate +1    
    for i in range(0,len(data)):
        if predict[i] == real[i]:
            accurate = accurate + 1
    accuracy = accurate / len(data)  
    return accuracy

In [14]:
def main():
    #get the train data
    train_data = load("C:/Users/11702/Desktop/大三下/CSC1001/Midterm/google-play-store-apps/train.csv")
    
    #create the tree
    tree = create_tree(train_data)

    #get the test data
    test_data = load("C:/Users/11702/Desktop/大三下/CSC1001/Midterm/google-play-store-apps/test.csv")
    
    #print the accuracy
    print('The accuracy is',str(test(test_data,tree)*100)[:5],'%')

In [15]:
main()

The accuracy is 80.36 %
